# LLMRails

>[LLMRails](https://www.llmrails.com/) is a API platform for building GenAI applications. It provides an easy-to-use API for document indexing and querying that is managed by LLMRails and is optimized for performance and accuracy. 
See the [LLMRails API documentation ](https://docs.llmrails.com/) for more information on how to use the API.

This notebook shows how to use functionality related to the `LLMRails`'s integration with langchain.
Note that unlike many other integrations in this category, LLMRails provides an end-to-end managed service for retrieval augmented generation, which includes:
1. A way to extract text from document files and chunk them into sentences.
2. Its own embeddings model and vector store - each text segment is encoded into a vector embedding and stored in the LLMRails internal vector store
3. A query service that automatically encodes the query into embedding, and retrieves the most relevant text segments (including support for [Hybrid Search](https://docs.llmrails.com/datastores/search))

All of these are supported in this LangChain integration.

# Setup

You will need a LLMRails account to use LLMRails with LangChain. To get started, use the following steps:
1. [Sign up](https://console.llmrails.com/signup) for a LLMRails account if you don't already have one.
2. Next you'll need to create API keys to access the API. Click on the **"API Keys"** tab in the corpus view and then the **"Create API Key"** button. Give your key a name. Click "Create key" and you now have an active API key. Keep this key confidential. 

To use LangChain with LLMRails, you'll need to have this value: api_key.
You can provide those to LangChain in two ways:

1. Include in your environment these two variables: `LLM_RAILS_API_KEY`, `LLM_RAILS_DATASTORE_ID`.

> For example, you can set these variables using os.environ and getpass as follows:

```python
import os
import getpass

os.environ["LLM_RAILS_API_KEY"] = getpass.getpass("LLMRails API Key:")
os.environ["LLM_RAILS_DATASTORE_ID"] = getpass.getpass("LLMRails Datastore Id:")
```

1. Provide them as arguments when creating the LLMRails vectorstore object:

```python
vectorstore = LLMRails(
    api_key=llm_rails_api_key,
    datastore_id=datastore_id
)
```

## Adding text

For adding text to your datastore first you have to go to [Datastores](https://console.llmrails.com/datastores) page and create one. Click Create Datastore button and choose a name and embedding model for your datastore. Then get your datastore id from newly created  datatore settings.
 

In [3]:
from langchain.vectorstores import LLMRails
import os

os.environ['LLM_RAILS_DATASTORE_ID'] = 'Your datastore id '
os.environ['LLM_RAILS_API_KEY'] = 'Your API Key'

llm_rails = LLMRails.from_texts(['Your text here'])

## Similarity search

The simplest scenario for using LLMRails is to perform a similarity search. 

In [4]:
query = "What do you plan to do about national security?"
found_docs = llm_rails.similarity_search(
    query, k=5
)

In [5]:
print(found_docs[0].page_content)

Others may not be democratic but nevertheless depend upon a rules-based international system.

Yet what we share in common, and the prospect of a freer and more open world, makes such a broad coalition necessary and worthwhile.

We will listen to and consider ideas that our partners suggest about how to do this.

Building this inclusive coalition requires reinforcing the multilateral system to uphold the founding principles of the United Nations, including respect for international law.

141 countries expressed support at the United Nations General Assembly for a resolution condemning Russia’s unprovoked aggression against Ukraine.

We continue to demonstrate this approach by engaging all regions across all issues, not in terms of what we are against but what we are for.

This year, we partnered with ASEAN to advance clean energy infrastructure and maritime security in the region.

We kickstarted the Prosper Africa Build Together Campaign to fuel economic growth across the continent an

## Similarity search with score

Sometimes we might want to perform the search, but also obtain a relevancy score to know how good is a particular result.

In [8]:
query = "What is your approach to national defense"
found_docs = llm_rails.similarity_search_with_score(
    query, k=5,
)

In [9]:
document, score = found_docs[0]
print(document.page_content)
print(f"\nScore: {score}")

But we will do so as the last resort and only when the objectives and mission are clear and achievable, consistent with our values and laws, alongside non-military tools, and the mission is undertaken with the informed consent of the American people.

Our approach to national defense is described in detail in the 2022 National Defense Strategy.

Our starting premise is that a powerful U.S. military helps advance and safeguard vital U.S. national interests by backstopping diplomacy, confronting aggression, deterring conflict, projecting strength, and protecting the American people and their economic interests.

Amid intensifying competition, the military’s role is to maintain and gain warfighting advantages while limiting those of our competitors.

The military will act urgently to sustain and strengthen deterrence, with the PRC as its pacing challenge.

We will make disciplined choices regarding our national defense and focus our attention on the military’s primary responsibilities: to

## LLMRails as a Retriever

LLMRails, as all the other LangChain vectorstores, is most often used as a LangChain Retriever:

In [10]:
retriever = llm_rails.as_retriever()
retriever

LLMRailsRetriever(tags=None, metadata=None, vectorstore=<langchain.vectorstores.llm_rails.LLMRails object at 0x107b9c040>, search_type='similarity', search_kwargs={'k': 5})

In [12]:
query = "What is your approach to national defense"
retriever.get_relevant_documents(query)[0]

Document(page_content='But we will do so as the last resort and only when the objectives and mission are clear and achievable, consistent with our values and laws, alongside non-military tools, and the mission is undertaken with the informed consent of the American people.\n\nOur approach to national defense is described in detail in the 2022 National Defense Strategy.\n\nOur starting premise is that a powerful U.S. military helps advance and safeguard vital U.S. national interests by backstopping diplomacy, confronting aggression, deterring conflict, projecting strength, and protecting the American people and their economic interests.\n\nAmid intensifying competition, the military’s role is to maintain and gain warfighting advantages while limiting those of our competitors.\n\nThe military will act urgently to sustain and strengthen deterrence, with the PRC as its pacing challenge.\n\nWe will make disciplined choices regarding our national defense and focus our attention on the milita